In [ ]:
'''In this notebook we will classify the prescence/abscence of HPO terms for 183,000 PubMed Abstracts using logistic 
regression and tf-idf features. We need the following files:

articles_hpo.txt : a map of articles to the appropriate HPO terms 
pmid_abstract.txt : a map of pmids and abstracts 

NOTE: pmid_abstract.txt includes articles that do not have an HPO term, so we 
will from the list of articles in articles_hpo.txt 

There are 1,307 HPO terms, so we will do at most 1,307 logistic regressions

Adapted in part from https://github.com/zygmuntz/classifying-text
'''

import numpy as np 
import re 
import os 
import pandas as pd 

from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.linear_model import LogisticRegression as LR
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_auc_score

#if the text files are somewhere else, change this 
data_path = os.getcwd()+'/data/'

articles_hpo_file = open(data_path+'articles_hpo.txt')
pmid_abstracts_file = open(data_path+'pmid_abstract.txt')

#make an array of pmids and set of HPO terms 
print('Parsing files...')
articles_hpo = {}
pmids = set() 
hpo_terms = set() 
for line in articles_hpo_file:
    parse = line.rstrip().split('\t')
    parse[0] = parse[0].replace('"','')
    articles_hpo[int(parse[0])] = '\t'.join(parse[1:])
    pmids.add(int(parse[0]))
    hpo_terms |= set(parse[1:])

#make a dictionary of abstracts 
pmid_abstracts = {}
for line in pmid_abstracts_file:
    parse = line.rstrip().split('\t')
    pmid_abstracts[int(parse[0])] = pmid_abstracts.get(parse[0],'')+'\t'.join(parse[1:])
    
#make sure we are only dealing with articles that have abstracts and HPO terms 
pmids = pmids.intersection(set(pmid_abstracts.keys()))

articles_hpo_file.close()
pmid_abstracts_file.close()
print('Creating DataFrame...')
#make a matrix of articles and hpo terms using Pandas DataFrame
df = pd.DataFrame(index=list(pmids), columns = list(hpo_terms))
df = df.fillna(0)
#fill in our dataframe 
for pmid in pmids: 
    for hpo_term in articles_hpo[pmid].split('\t'):
        df.loc[pmid,hpo_term] = 1

#array that we will shuffle for test/train splits 
indices = np.arange(len(pmids))

print('Working with {} articles and {} hpo terms'.format(len(pmids),len(hpo_terms)))

In [1]:
#Function to clean and add an abstract to our list of clean abstracts 
def abstract_to_words(abstract):
    #remove non words 
    clean_abstract = re.sub("[^a-zA-Z0-9]"," ", abstract)
    #split on the words 
    return(clean_abstract.lower().split())

In [ ]:
#generate train/test split 
train_index, test_index = train_test_split(indices, test_size=.2)

train_pmids = df.index[train_index]
test_pmids = df.index[test_index]

train_abstracts = []
test_abstracts = []
print('Generating train/test split...')

for pmid in train_pmids:
    train_abstracts.append(" ".join(abstract_to_words(pmid_abstracts[pmid])))
for pmid in test_pmids:
    test_abstracts.append(" ".join(abstract_to_words(pmid_abstracts[pmid])))

print('Vectorizing...')

FEATURES = 100
vectorizer = TfidfVectorizer( max_features = FEATURES , ngram_range = ( 1, 3 ), sublinear_tf = True )

train_x = vectorizer.fit_transform(train_abstracts)
test_x = vectorizer.fit_transform(test_abstracts)

models = {}
THRESHOLD = 100 
scores = {}
auc = {}
for hpo in df.columns: 
    if np.sum(df.loc[:,hpo].values)>THRESHOLD:
        train_labels = df.loc[:,hpo].values[train_index]
        test_labels = df.loc[:,hpo].values[test_index]
        model = LR(class_weight='balanced')
        print('Training model for HPO term {}'.format(hpo))
        model.fit( train_x, train_labels)
        proba = model.predict_proba(test_x)[:,1]
        score = model.score(test_x, test_labels)
        pos_score = model.score(test_x[test_labels>0],test_labels[test_labels>0])
        auc_score = roc_auc_score(test_labels,proba)
        print('Accuracy: {} Positive Acc: {} AUC: {}'.format(score,pos_score,auc_score))
        scores[hpo] = score
        auc[hpo] = auc_score
        models[hpo] = model  
    else:
        print("HPO term {} does not meet threshold".format(hpo))